# Logistic regression from scratch

## Objectives:
- Implement the link function for logistic regression.
- Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
- Implement gradient ascent.
- Compute classification accuracy for the logistic regression model.

## Import library and load data

In [1]:
from graphlab import SFrame

In [2]:
products = SFrame('./amazon_baby_subset.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to bzha0010@student.monsh.edu and will expire on June 12, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\zby0902\AppData\Local\Temp\graphlab_server_1531817637.log.0


## EDA

In [3]:
products.head(5)

name,review,rating,sentiment
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,We wanted to getsomething to keep track ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,My daughter had her 1stbaby over a year ago. ...,5.0,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,Very cute interactivebook! My son loves this ...,5.0,1


In [4]:
#number of positive and negative sentiment
products[products['sentiment'] == 1].shape[0], products.shape[0] - products[products['sentiment'] == 1].shape[0]

(26579, 26493)

### Load important words

In [5]:
import json

In [6]:
with open('important_words.json','r') as f:
    important_words = json.load(f)

In [7]:
important_words = [str(s) for s in important_words]

### Clean reviews

In [8]:
products.fillna('review','');

In [9]:
def remove_punc(text):
    import string
    return text.translate(None, string.punctuation)

In [10]:
products['review_clean'] = products['review'].apply(remove_punc)

### Generate word count

Before generating the word_counts of all important words we need to tranform all reviews to lower case.

In [11]:
## Should unify the upper and lower case to avoid missed count
#products['review_clean'] = products['review_clean'].apply(lambda s: s.lower() )

Then we generate the count for each importan word

In [14]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s: s.split().count(word))

``Quiz:`` No of reviews contains 'perfect'

In [150]:
(products['perfect'] != 0).sum()

2955L

In [20]:
def get_feature_matrix(df,features,label):
    """
    Extract the feature matrix and the target matrix from the dataframe
    
    Parameters
    ----------
    df: The dataframe(SFrame or Dataframe) containing the data points
    
    features: list of features picked to form the feature matrix
    
    label: The target feature of the model
    
    Return
    ------
    feature_matrix : A 2D array containing all features for all data points
    
    class_matrix : A 1D array containing the class labels for all instances
    """
    df['intercept'] = 1
    features = ['intercept'] + features
    feature_matrix = df[features].to_numpy()
    class_matrix = df[label].to_numpy()
        
    return feature_matrix, class_matrix

In [21]:
import numpy as np

In [22]:
feature_matrix, sentiment = get_feature_matrix(products, important_words, 'sentiment')

###  The predict method

In [88]:
def predict_probability(feature_matrix, weights):
    """
    Predict the probabilitie of each instances as class 1
    
    Parameters
    ----------
    feature_matrix: The 2D array containing all selected features of all data points
    
    weights: The learnt coefficients of the linear score function
    
    Return
    ------
    prob: The probility of all instances as class 1
    """
    scores = np.dot(feature_matrix,weights)
    prob = 1 / (1+np.exp(-scores))
    
    return prob

## Checkpoint: A test of the previous methods

In [89]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print 'The following outputs must match '
print '------------------------------------------------'
print 'correct_predictions           =', correct_predictions
print 'output of predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients)

The following outputs must match 
------------------------------------------------
correct_predictions           = [ 0.98201379  0.26894142]
output of predict_probability = [ 0.98201379  0.26894142]


#### The derivative for single feature 

In [90]:
def feature_derivative(errors, feature):
    """
    compute the derivative of the log-likelihood with respect to a single feature
    
    Parameters
    ----------
    errors: A 1D array containing differences between the boolean of the instance as 1 and its probility to be 1
    
    feature: A 1D array containing values of a single feature for all data points
    
    Return
    ------
    The derivative of the log-likelihood, which is the derivative with respect to a single coefficient w_j. 
    """
    return np.dot(errors,feature)

### The log-likelihood function

In [96]:
def compute_log_likelihood(feature_matrix,sentiment,weights):
    """
    compute the log-likelihood of all features.
    
    Parameters
    ----------
    feature_matrix: A 2D array containing all selected featuers of all instances
    
    sentiment: A 1D array containing class lables of all instances
    
    weights: The coefficients of all features
    
    Return
    ------
    log_like : The log likelihood for the current iteration
    """
    indicator = sentiment == 1
    scores = np.dot(feature_matrix,weights)
    log_like = np.sum((indicator-1)*scores - np.log(1.+np.exp(-scores)))
    
    return log_like

## Test the derivative related methods

In [97]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll = sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print 'The following outputs must match '
print '------------------------------------------------'
print 'correct_log_likelihood           =', correct_ll
print 'output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients)

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.33141161544
output of compute_log_likelihood = -5.33141161544


### Implement of logistic regression

In [99]:
from math import sqrt

In [113]:
def logistic_regression(feature_matrix, sentiment, 
                        initial_coefficients, step_size, max_iter):
    """
    learn the optimal weights for the logistic regression model using gradiend ascent.
    
    Parameters
    ----------
    feature_matrix: A 2D array of features for all instances
    
    sentimnet: A 1D array of true class labels
    
    initial_coefficients: The initial weights for all features
    
    step_size: The step_size for gradient ascent
    
    max_iter: The limit number of iterations
    
    Return
    ------
    The final coefficents for all features(a 1D array)
    """
    weights = np.array(initial_coefficients)
    
    for itr in range(max_iter):
        prediction = predict_probability(feature_matrix, weights)
        indicator = sentiment == 1
        errors = indicator - prediction
        
        for j in range(len(weights)):
            derivative = feature_derivative(errors, feature_matrix[:,j])
            weights[j] += step_size * derivative
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, weights)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)      
    return weights

### Train the model to learn the weights

In [239]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

## （Own test of tune step_size）

In [228]:
coefficients3 = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=1e-4, max_iter=301)

iteration   0: log likelihood of observed labels = -32421.40457758
iteration   1: log likelihood of observed labels = -32361.44206679
iteration   2: log likelihood of observed labels = -46284.52561600
iteration   3: log likelihood of observed labels = -76428.62370127
iteration   4: log likelihood of observed labels = -78302.76681108
iteration   5: log likelihood of observed labels = -61756.91248927
iteration   6: log likelihood of observed labels = -65412.29017649
iteration   7: log likelihood of observed labels = -52088.67904541
iteration   8: log likelihood of observed labels = -52970.08854271
iteration   9: log likelihood of observed labels = -45218.93727170
iteration  10: log likelihood of observed labels = -45361.30108853
iteration  11: log likelihood of observed labels = -40648.70439891
iteration  12: log likelihood of observed labels = -40837.87419519
iteration  13: log likelihood of observed labels = -37737.60805148
iteration  14: log likelihood of observed labels = -38096.9019

In [227]:
## own experiment on step_size
coefficients2 = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=1e-5, max_iter=301)

iteration   0: log likelihood of observed labels = -36222.96935992
iteration   1: log likelihood of observed labels = -35712.39105128
iteration   2: log likelihood of observed labels = -35244.87859896
iteration   3: log likelihood of observed labels = -34814.71047654
iteration   4: log likelihood of observed labels = -34417.49249984
iteration   5: log likelihood of observed labels = -34049.53704724
iteration   6: log likelihood of observed labels = -33707.68013590
iteration   7: log likelihood of observed labels = -33389.18706537
iteration   8: log likelihood of observed labels = -33091.68390021
iteration   9: log likelihood of observed labels = -32813.10213697
iteration  10: log likelihood of observed labels = -32551.63288262
iteration  11: log likelihood of observed labels = -32305.68866325
iteration  12: log likelihood of observed labels = -32073.87149461
iteration  13: log likelihood of observed labels = -31854.94611400
iteration  14: log likelihood of observed labels = -31647.8174

In [235]:
coefficients4 = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=8e-5, max_iter=301)

iteration   0: log likelihood of observed labels = -33086.51747495
iteration   1: log likelihood of observed labels = -31881.46236863
iteration   2: log likelihood of observed labels = -34970.52252035
iteration   3: log likelihood of observed labels = -46563.94189047
iteration   4: log likelihood of observed labels = -58698.17681138
iteration   5: log likelihood of observed labels = -50519.96779930
iteration   6: log likelihood of observed labels = -51655.45860328
iteration   7: log likelihood of observed labels = -43137.69324957
iteration   8: log likelihood of observed labels = -43025.34336656
iteration   9: log likelihood of observed labels = -37996.53459829
iteration  10: log likelihood of observed labels = -37510.65043503
iteration  11: log likelihood of observed labels = -34495.37279778
iteration  12: log likelihood of observed labels = -34073.63979852
iteration  13: log likelihood of observed labels = -32174.27038303
iteration  14: log likelihood of observed labels = -31878.2823

In [212]:
products['predic_prob1'] = predict_probability(feature_matrix,coefficients)
products['predic_prob2'] = predict_probability(feature_matrix,coefficients2)

### Predict sentiment using the learnt coefficients

In [240]:
scores = np.dot(feature_matrix,coefficients)
scores2 = np.dot(feature_matrix,coefficients2)
scores4 = np.dot(feature_matrix,coefficients4)

In [241]:
products['predic_scores1'] = scores
products['predic_scores2'] = scores2
products['predic_scores4'] = scores4

In [242]:
products['predic_sentiment1'] = products['predic_scores1'].apply(lambda s: 1 if s>0 else -1)
products['predic_sentiment2'] = products['predic_scores2'].apply(lambda s: 1 if s>0 else -1)
products['predic_sentiment4'] = products['predic_scores4'].apply(lambda s: 1 if s>0 else -1)

In [243]:
# No of positive_predicted instances
(products['predic_sentiment1'] == 1).sum(),(products['predic_sentiment2'] == 1).sum()

(25126L, 25679L)

In [248]:
# accuracy of coefficients and coefficients2
round((products['predic_sentiment1'] == products['sentiment']).sum()*1./products.shape[0],2),round((products['predic_sentiment2'] == products['sentiment']).sum()*1./products.shape[0],4)

(0.75, 0.7877)

In [247]:
# accuracy of coefficients4 which is believed to be opitimum regarding the step_size
round((products['predic_sentiment4'] == products['sentiment']).sum()*1./products.shape[0],4)

0.7923

### Find the most positive and negative words

In [249]:
coefficients = list(coefficients)[1:]
word_coefficient_paris = [(word,coef) for word,coef in zip(important_words,coefficients)]


In [250]:
word_coef_dict = {a:b for a,b in zip(important_words,coefficients)}

In [251]:
word_coef_dict['work']

-0.033069515294752737

#### Ten most positive words

In [252]:
sorted(word_coefficient_paris,key=lambda x: x[1],reverse=True)[:10]

[('great', 0.066546084170457695),
 ('love', 0.065890762922123258),
 ('easy', 0.06479458680257838),
 ('little', 0.045435626308421372),
 ('loves', 0.044976401394906038),
 ('well', 0.030135001092107074),
 ('perfect', 0.029739937104968462),
 ('old', 0.020077541034775381),
 ('nice', 0.018408707995268992),
 ('daughter', 0.01770319990570169)]

#### Ten most negative words

In [253]:
sorted(word_coefficient_paris,key=lambda x: x[1])[:10]

[('would', -0.053860148445203121),
 ('product', -0.041511033392108897),
 ('money', -0.038982037286487109),
 ('work', -0.033069515294752737),
 ('even', -0.030051249236035808),
 ('disappointed', -0.028978976142317068),
 ('get', -0.028711552980192581),
 ('back', -0.027742697230661331),
 ('return', -0.026592778462247283),
 ('monitor', -0.024482100545891717)]